### 2021 CSD

In [1]:
import duckdb
import pandas

# Connect to DuckDB (it will create a database file in the current directory)
conn = duckdb.connect('can.db')

# Load CSV data into a DuckDB table
conn.execute(f"""
    DROP TABLE csd2021_2016;
    CREATE TABLE csd2021_2016 AS 
    SELECT CENSUS_YEAR,
        ALT_GEO_CODE,
        GEO_NAME,
        CHARACTERISTIC_ID,
        CHARACTERISTIC_NAME,
        C1_COUNT_TOTAL,
        C10_RATE_TOTAL ,
    FROM csd2021
    WHERE
             ALT_GEO_CODE = 1102075
             OR
             ALT_GEO_CODE = 1209034
             OR
             ALT_GEO_CODE = 5917034
             OR
             ALT_GEO_CODE = 1310032
             OR
             ALT_GEO_CODE = 1307045
             OR
             ALT_GEO_CODE = 1307022
             ;
""")

# Close the connection
conn.close()

In [2]:
# Connect to the DuckDB database
conn = duckdb.connect('can.db')

result = conn.execute("SELECT DISTINCT GEO_NAME FROM csd2021_2016").fetchall()
for row in  result:
    print(row)

# SQL command to get the column names of csd2021_2016 table
get_columns_query = """
PRAGMA table_info(csd2021_2016)
"""

# Execute the query
result = conn.execute(get_columns_query).fetchall()

# Extract and print the column names
column_names = [row[1] for row in result]  # The second element in each row contains the column names
print(column_names)


# Close the connection
conn.close()

('Charlottetown, City (CY)',)
('Halifax, Regional municipality (RGM)',)
('Moncton, City (C)',)
('Dieppe, City (C)',)
('Fredericton, City (C)',)
('Victoria, City (CY)',)
['CENSUS_YEAR', 'ALT_GEO_CODE', 'GEO_NAME', 'CHARACTERISTIC_ID', 'CHARACTERISTIC_NAME', 'C1_COUNT_TOTAL', 'C10_RATE_TOTAL']


# 2016

In [3]:
import duckdb

conn.close()

# connnect to the DuckDB database
conn = duckdb.connect(database='can.db')

# SQL query to fetch the required data
fetch_query = """
SELECT
    CENSUS_YEAR,
    ALT_GEO_CODE,
    GEO_NAME,
    CHARACTERISTIC_ID, 
    CHARACTERISTIC_NAME,
    total AS C1_COUNT_TOTAL
FROM csd2016
WHERE

    ALT_GEO_CODE = 1102075
    OR ALT_GEO_CODE = 1209034
    OR ALT_GEO_CODE = 5917034
    OR ALT_GEO_CODE = 1310032
    OR ALT_GEO_CODE = 1307045
    OR ALT_GEO_CODE = 1307022
"""

# Fetch the data
data = conn.execute(fetch_query).fetchall()

# Insert the data into csd2021_2016 table
insert_query = """
INSERT INTO csd2021_2016 (CENSUS_YEAR, ALT_GEO_CODE, GEO_NAME, CHARACTERISTIC_ID, CHARACTERISTIC_NAME, C1_COUNT_TOTAL)
VALUES (?, ?, ?, ?, ?, ?)
"""

# Execute the insertion for each row of data
conn.executemany(insert_query, data)

# Verify the insertion
verify_query = """
    SELECT *
    FROM csd2021_2016
    WHERE CENSUS_YEAR = 2016
    LIMIT 10
    """
result = conn.execute(verify_query).fetchall()
print("result:")
for row in result:
    print(row)


# Close the connnection
conn.close()


result:
(2016, '1102075', 'Charlottetown', 1, 'Population, 2016', 36094.0, None)
(2016, '1102075', 'Charlottetown', 2, 'Population, 2011', 34562.0, None)
(2016, '1102075', 'Charlottetown', 3, 'Population percentage change, 2011 to 2016', 4.400000095367432, None)
(2016, '1102075', 'Charlottetown', 4, 'Total private dwellings', 17193.0, None)
(2016, '1102075', 'Charlottetown', 5, 'Private dwellings occupied by usual residents', 16100.0, None)
(2016, '1102075', 'Charlottetown', 6, 'Population density per square kilometre', 814.0999755859375, None)
(2016, '1102075', 'Charlottetown', 7, 'Land area in square kilometres', 44.34000015258789, None)
(2016, '1102075', 'Charlottetown', 8, 'Total - Age groups and average age of the population - 100% data', 36095.0, None)
(2016, '1102075', 'Charlottetown', 9, '0 to 14 years', 5010.0, None)
(2016, '1102075', 'Charlottetown', 10, '0 to 4 years', 1585.0, None)


In [4]:
# Path to your CSV file
csv_file_path = '/home/mac/code/municipalities/2016-2021-correspondance.csv'

# Connect to DuckDB (it will create a database file in the current directory)
conn = duckdb.connect('can.db')

# Load CSV data into a DuckDB table
conn.execute(f"""
    DROP TABLE char1621;
    CREATE TABLE char1621 AS 
    SELECT * 
    FROM read_csv_auto('{csv_file_path}')
""")

# Verify that the table has been created and contains data
result = conn.execute("SELECT * FROM char1621 LIMIT 2").fetchall()
print(result)

# Use DESCRIBE to get the schema of the table
describe_result = conn.execute("DESCRIBE char1621").fetchall()
print("DESCRIBE result:")
for row in describe_result:
    print(row)

# Close the connection
conn.close()

[(1, 1, 'Population, 2021', 'Population', 'Population and dwellings'), (9, 9, '0 to 14 years', '0 à 14 ans', 'Age characteristics')]
DESCRIBE result:
('c2016', 'BIGINT', 'YES', None, None, None)
('c2021', 'BIGINT', 'YES', None, None, None)
('Characteristic', 'VARCHAR', 'YES', None, None, None)
('Caractéristique', 'VARCHAR', 'YES', None, None, None)
('Topic', 'VARCHAR', 'YES', None, None, None)


Duplicates

SELECT c2016
FROM can.main.char1621
GROUP BY c2016
HAVING COUNT(c2016) > 1;

In [5]:
# Connect to the DuckDB database
conn = duckdb.connect(database='can.db')

conn.execute("""
    ALTER TABLE csd2021_2016
    ADD COLUMN char2021 BIGINT;

    UPDATE csd2021_2016
    SET char2021 = CASE 
        WHEN CENSUS_YEAR = 2021 THEN CHARACTERISTIC_ID
        WHEN CENSUS_YEAR = 2016 THEN (
            SELECT char1621.c2021
            FROM char1621
            WHERE char1621.c2016 = csd2021_2016.CHARACTERISTIC_ID
        )
    END;
""")

# Verify that the table has been created and contains data
result = conn.execute("SELECT * FROM csd2021_2016 WHERE CENSUS_YEAR = 2016 LIMIT 2").fetchall()
print(result)

# Use DESCRIBE to get the schema of the table
describe_result = conn.execute("DESCRIBE char1621").fetchall()
print("DESCRIBE result:")
for row in describe_result:
    print(row)

# Close the connection
conn.close()

[(2016, '1102075', 'Charlottetown', 1, 'Population, 2016', 36094.0, None, 1), (2016, '1102075', 'Charlottetown', 2, 'Population, 2011', 34562.0, None, None)]
DESCRIBE result:
('c2016', 'BIGINT', 'YES', None, None, None)
('c2021', 'BIGINT', 'YES', None, None, None)
('Characteristic', 'VARCHAR', 'YES', None, None, None)
('Caractéristique', 'VARCHAR', 'YES', None, None, None)
('Topic', 'VARCHAR', 'YES', None, None, None)
